# ExoNAMD API v1.0

## Summary

The observer wants to compute the relative and/or absolute NAMD of:
- a given multiplanetary system;
- a subset of multiplanetary systems;
- or all the known ones.

This tool handles all of the above.

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from datetime import timedelta

pd.options.display.max_columns = 20
pd.options.display.max_rows = 30
pd.options.mode.copy_on_write = True
import warnings

warnings.filterwarnings("ignore")

from exonamd.utils import ROOT

### Task 1: getting the data

This task retrieves the system parameters from the NASA Exoplanet Archive database, and stores them in a local database.

In [8]:
# Define the URL for the API
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"

# Define the date you want to filter by
from_scratch = False
if from_scratch:
    latest = datetime.strptime('1950-01-01', "%Y-%m-%d")  # Example date
else:
    df_latest = pd.read_csv(os.path.join(ROOT, "data", "exo.csv"))
    latest = df_latest['rowupdate'].max()
    latest = datetime.strptime(latest, "%Y-%m-%d")  # Example date
    latest = latest - timedelta(days=1)

# Convert the date to a string in the format 'YYYY-MM-DD'
latest = latest.strftime('%Y-%m-%d')

# Define the SQL query to retrieve the required data
query = f"""
SELECT 
    hostname, 
    pl_name, 
    default_flag,
    rowupdate,
    sy_pnum, 
    pl_orbsmax, 
    pl_orbsmaxerr1, 
    pl_orbsmaxerr2, 
    pl_rade,
    pl_radeerr1,
    pl_radeerr2,
    pl_bmasse, 
    pl_bmasseerr1, 
    pl_bmasseerr2, 
    pl_orbeccen, 
    pl_orbeccenerr1, 
    pl_orbeccenerr2, 
    pl_orbincl, 
    pl_orbinclerr1, 
    pl_orbinclerr2,
    pl_trueobliq,
    pl_trueobliqerr1,
    pl_trueobliqerr2
FROM ps
WHERE
    sy_pnum > 1
    AND rowupdate > '{latest}'
"""

# Define the parameters for the request
params = {
    "query": query,
    "format": "json",
}

# Make the request to the API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Convert the JSON data to a pandas DataFrame
    df_query = pd.DataFrame(data)
else:
    print(f"Error: {response.status_code}")

In [9]:
df_new = df_query.copy()

if from_scratch:
    df = df_new
else:
    df = pd.concat([df_new, df_latest], ignore_index=True)
    df = df.drop_duplicates(keep='last')

df.to_csv(os.path.join(ROOT, "data", "exo.csv"), index=False)

In [10]:
df

,hostname,pl_name,default_flag,rowupdate,sy_pnum,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_rade,pl_radeerr1,...,pl_bmasseerr2,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_trueobliq,pl_trueobliqerr1,pl_trueobliqerr2
6,BD-08 2823,BD-08 2823 b,0,2018-04-25,2,0.0560,0.002,-0.002,NaN,NaN,...,-2.1000,0.150,0.150,-0.150,NaN,NaN,NaN,NaN,NaN,NaN
7,BD-08 2823,BD-08 2823 c,1,2014-05-14,2,0.6800,0.020,-0.020,NaN,NaN,...,-10.0000,0.190,0.090,-0.090,NaN,NaN,NaN,NaN,NaN,NaN
8,HR 8799,HR 8799 c,1,2014-05-14,4,38.0000,NaN,NaN,13.00,1.00,...,-1000.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,HD 47366,HD 47366 c,0,2019-09-20,2,1.8530,0.045,-0.045,NaN,NaN,...,-47.6745,0.278,0.067,-0.094,NaN,NaN,NaN,NaN,NaN,NaN
10,gam Lib,gam Lib b,0,2018-04-25,2,1.2400,0.100,-0.100,NaN,NaN,...,-44.4962,0.210,0.100,-0.100,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16967,Kepler-1996,Kepler-1996 c,1,2017-05-08,2,0.3608,NaN,NaN,2.81,0.20,...,NaN,0.000,NaN,NaN,89.19,NaN,NaN,NaN,NaN,NaN
16968,Kepler-1991,Kepler-1991 c,0,2014-11-18,2,0.1549,NaN,NaN,2.18,0.75,...,NaN,0.000,NaN,NaN,89.95,NaN,NaN,NaN,NaN,NaN
16969,Kepler-1987,Kepler-1987 d,0,2016-07-26,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16970,Kepler-2001,Kepler-2001 c,1,2017-05-08,2,0.1084,NaN,NaN,1.82,0.23,...,NaN,0.000,NaN,NaN,88.45,NaN,NaN,NaN,NaN,NaN
